<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Local Automated Machine Learning Model with ACI Deployment for Predicting Sentence Similarity

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/nlp/examples/sentence_similarity/automl_local_deployment_aci.png)

This notebook demonstrates how to use [Azure Machine Learning Service's](https://azure.microsoft.com/en-us/services/machine-learning-service/
) Automated Machine Learning ([AutoML](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-automated-ml
)) locally to automate machine learning model selection and tuning and how to use Azure Container Instance ([ACI](https://azure.microsoft.com/en-us/services/container-instances/
)) for deployment. We utilize the STS Benchmark dataset to predict sentence similarity and utilize AutoML's text preprocessing features.

## Table of Contents
1. [Introduction](#1.-Introduction)  
     [1.1 What is Azure AutoML?](#1.1-What-is-Azure-AutoML?)  
     [1.2 Modeling Problem](#1.2-Modeling-Problem)  
1. [Data Preparation](#2.-Data-Preparation)  
1. [Create AutoML Run](#3.-Create-AutoML-Run)    
    [3.1 Link to or create a Workspace](#3.1-Link-to-or-create-a-Workspace)  
    [3.2 Create AutoMLConfig object](#3.2-Create-AutoMLConfig-object)
    [3.3 Run Experiment](#3.3-Run-Experiment)      
1. [Deploy Sentence Similarity Model](#4.-Deploy-Sentence-Similarity-Model)  
    [4.1 Retrieve the Best Model](#4.1-Retrieve-the-Best-Model)  
    [4.2 Register the Fitted Model for Deployment](#4.2-Register-the-Fitted-Model-for-Deployment)   
    [4.3 Create an Entry Script](#4.3-Create-an-Entry-Script)   
    [4.4 Create a YAML File for the Environment](#4.4-Create-a-YAML-File-for-the-Environment)  
    [4.5 Create a Container Image](#4.5-Create-a-Container-Image)    
    [4.6 Deploy the Image as a Web Service to Azure Container Instance](#4.6-Deploy-the-Image-as-a-Web-Service-to-Azure-Container-Instance)  
    [4.7 Test Deployed Model](#4.7-Test-Deployed-Model)     
1. [Clean](#5-Clean)

### 1.1 What is Azure AutoML?

Automated machine learning ([AutoML](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-automated-ml)) is a capability of Microsoft's [Azure Machine Learning service](https://azure.microsoft.com/en-us/services/machine-learning-service/
). The goal of AutoML is to improve the productivity of data scientists and democratize AI by allowing for the rapid development and deployment of machine learning models. To acheive this goal, AutoML automates the process of selecting a ML model and tuning the model. All the user is required to provide is a dataset (suitable for a classification, regression, or time-series forecasting problem) and a metric to optimize in choosing the model and hyperparameters. The user is also given the ability to set time and cost constraints for the model selection and tuning.

![](https://nlpbp.blob.core.windows.net/images/automl.PNG)

The AutoML model selection and tuning process can be easily tracked through the Azure portal or directly in python notebooks through the use of widgets. AutoML quickly selects a high quilty machine learning model tailored for your prediction problem. In this notebook, we walk through the steps of preparing data, setting up an AutoML experiment, and evaluating the results of our best model. More information about running AutoML experiments in Python can be found [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train). 

### 1.2 Modeling Problem

The regression problem we will demonstrate is predicting sentence similarity scores on the STS Benchmark dataset. The [STS Benchmark dataset](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark#STS_benchmark_dataset_and_companion_dataset) contains a selection of English datasets that were used in Semantic Textual Similarity (STS) tasks 2012-2017. The dataset contains 8,628 sentence pairs with a human-labeled integer representing the sentences' similarity (ranging from 0, for no meaning overlap, to 5, meaning equivalence). The sentence pairs will be embedded using AutoML's built-in preprocessing, so we'll pass the sentences directly into the model.

In [1]:
# Set the environment path to find NLP
import sys

sys.path.append("../../")
import time
import os
import pandas as pd
import shutil
import numpy as np
import torch
import sys
from scipy.stats import pearsonr
from scipy.spatial import distance
from sklearn.externals import joblib
import json
import scrapbook as sb

# Import utils
from utils_nlp.azureml import azureml_utils
from utils_nlp.dataset import stsbenchmark
from utils_nlp.dataset.preprocess import (
    to_lowercase,
    to_spacy_tokens,
    rm_spacy_stopwords,
)
from utils_nlp.common.timer import Timer

# Tensorflow dependencies for Google Universal Sentence Encoder
import tensorflow_hub as hub


# AzureML packages
import azureml as aml
import logging
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails
from azureml.train.automl.run import AutoMLRun
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage
from azureml.core.conda_dependencies import CondaDependencies

print("System version: {}".format(sys.version))
print("Azure ML SDK Version:", aml.core.VERSION)
print("Pandas version: {}".format(pd.__version__))

Turning diagnostics collection on. 
System version: 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
Azure ML SDK Version: 1.0.48
Pandas version: 0.23.4


In [2]:
BASE_DATA_PATH = "../../data"
CPU_CORES = 1
MEMORY_GB = 8

# Define the settings for AutoML
automl_task = "regression"
automl_iteration_timeout = 15
automl_iterations = 50
automl_metric = "spearman_correlation"
automl_preprocess = True
automl_model_blacklist = ['XGBoostRegressor']

config_path = (
    "./.azureml"
)  # Path to the directory containing config.json with azureml credentials

webservice_name = "aci-automl-service" #name for webservice; must be unique within your workspace

# Azure resources
subscription_id = "YOUR_SUBSCRIPTION_ID"
resource_group = "YOUR_RESOURCE_GROUP_NAME"  
workspace_name = "YOUR_WORKSPACE_NAME"  
workspace_region = "YOUR_WORKSPACE_REGION" #Possible values eastus, eastus2 and so on.

In [3]:
automl_settings = {
    "task": automl_task,  # type of task: classification, regression or forecasting
    "debug_log": "automated_ml_errors.log",
    "path": "./automated-ml-regression",
    "iteration_timeout_minutes": automl_iteration_timeout,  # How long each iteration can take before moving on
    "iterations": automl_iterations,  # Number of algorithm options to try
    "primary_metric": automl_metric,  # Metric to optimize
    "preprocess": automl_preprocess,  # Whether dataset preprocessing should be applied
    "blacklist_models": automl_model_blacklist #exclude this model due to installation issues
}

# 2. Data Preparation

## STS Benchmark Dataset

As described above, the STS Benchmark dataset contains 8.6K sentence pairs along with a human-annotated score for how similiar the two sentences are. We will load the training, development (validation), and test sets provided by STS Benchmark and preprocess the data (lowercase the text, drop irrelevant columns, and rename the remaining columns) using the utils contained in this repo. Each dataset will ultimately have three columns: _sentence1_ and _sentence2_ which contain the text of the sentences in the sentence pair, and _score_ which contains the human-annotated similarity score of the sentence pair.

In [4]:
# Load in the raw datasets as pandas dataframes
train_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="train")
dev_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="dev")
test_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="test")

100%|██████████████████████████████████████████████████████████████████████████████████| 401/401 [00:02<00:00, 146KB/s]


Data downloaded to ../../data\raw\stsbenchmark


100%|██████████████████████████████████████████████████████████████████████████████████| 401/401 [00:02<00:00, 172KB/s]


Data downloaded to ../../data\raw\stsbenchmark


100%|██████████████████████████████████████████████████████████████████████████████████| 401/401 [00:02<00:00, 182KB/s]


Data downloaded to ../../data\raw\stsbenchmark


In [5]:
# Clean each dataset by lowercasing text, removing irrelevant columns,
# and renaming the remaining columns
train_clean = stsbenchmark.clean_sts(train_raw)
dev_clean = stsbenchmark.clean_sts(dev_raw)
test_clean = stsbenchmark.clean_sts(test_raw)

In [6]:
# Convert all text to lowercase
train = to_lowercase(train_clean)
dev = to_lowercase(dev_clean)
test = to_lowercase(test_clean)

In [7]:
print("Training set has {} sentences".format(len(train)))
print("Development set has {} sentences".format(len(dev)))
print("Testing set has {} sentences".format(len(test)))

Training set has 5749 sentences
Development set has 1500 sentences
Testing set has 1379 sentences


In [8]:
train.head()

,score,sentence1,sentence2
0,5.00,a plane is taking off.,an air plane is taking off.
1,3.80,a man is playing a large flute.,a man is playing a flute.
2,3.80,a man is spreading shreded cheese on a pizza.,a man is spreading shredded cheese on an uncoo...
3,2.60,three men are playing chess.,two men are playing chess.
4,4.25,a man is playing the cello.,a man seated is playing the cello.


# 3. Create AutoML Run

AutoML can be used for classification, regression or timeseries experiments. Each experiment type has corresponding machine learning models and metrics that can be optimized (see [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train)) and the options will be delineated below. As a first step we connect to an existing workspace or create one if it doesn't exist.

## 3.1 Link to or create a Workspace

The following cell looks to set up the connection to your [Azure Machine Learning service Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace). You can choose to connect to an existing workspace or create a new one. 

**To access an existing workspace:**
1. If you have a `config.json` file, you do not need to provide the workspace information; you will only need to update the `config_path` variable that is defined above which contains the file.
2. Otherwise, you will need to supply the following:
    * The name of your workspace
    * Your subscription id
    * The resource group name

**To create a new workspace:**

Set the following information:
* A name for your workspace
* Your subscription id
* The resource group name
* [Azure region](https://azure.microsoft.com/en-us/global-infrastructure/regions/) to create the workspace in, such as `eastus2`. 

This will automatically create a new resource group for you in the region provided if a resource group with the name given does not already exist. 

In [9]:
ws = azureml_utils.get_or_create_workspace(
    config_path=config_path,
    subscription_id=subscription_id,
    resource_group=resource_group,
    workspace_name=workspace_name,
    workspace_region=workspace_region,
)

Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
WARNING - You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.


In [10]:
print(
    "Workspace name: " + ws.name,
    "Azure region: " + ws.location,
    "Subscription id: " + ws.subscription_id,
    "Resource group: " + ws.resource_group,
    sep="\n",
)

## 3.2 Create AutoMLConfig object
Next, we specify the parameters for the AutoMLConfig class. 

**task**  
AutoML supports the following base learners for the regression task: Elastic Net, Light GBM, Gradient Boosting, Decision Tree, K-nearest Neighbors, LARS Lasso, Stochastic Gradient Descent, Random Forest, Extremely Randomized Trees, XGBoost, DNN Regressor, Linear Regression. In addition, AutoML also supports two kinds of ensemble methods: voting (weighted average of the output of multiple base learners) and stacking (training a second "metalearner" which uses the base algorithms' predictions to predict the target variable). Specific base learners can be included or excluded in the parameters for the AutoMLConfig class (whitelist_models and blacklist_models) and the voting/stacking ensemble options can be specified as well (enable_voting_ensemble and enable_stack_ensemble)

**preprocess**  
AutoML also has advanced preprocessing methods, eliminating the need for users to perform this manually. Data is automatically scaled and normalized but an additional parameter in the AutoMLConfig class enables the use of more advanced techniques including imputation, generating additional features, transformations, word embeddings, etc. (full list found [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-create-portal-experiments#preprocess)). Note that algorithm-specific preprocessing will be applied even if preprocess=False. 

**primary_metric**  
The regression metrics available are the following: Spearman Correlation (spearman_correlation), Normalized RMSE (normalized_root_mean_squared_error), Normalized MAE (normalized_mean_absolute_error), and R2 score (r2_score) 

**Constraints:**  
There is a cost_mode parameter to set cost prediction modes (see options [here](https://docs.microsoft.com/en-us/python/api/azureml-train-automl/azureml.train.automl.automlconfig?view=azure-ml-py)). To set constraints on time there are multiple parameters including experiment_exit_score (target score to exit the experiment after achieving), experiment_timeout_minutes (maximum amount of time for all combined iterations), and iterations (total number of different algorithm and parameter combinations to try).

**Note**: we are directly passing in sentence pairs as data because we are relying upon AutoML's built-in preprocessing (by setting preprocess = True in the AutoMLConfig parameters) to perform the embedding step.

In [11]:
X_train = train.drop("score", axis=1).values
y_train = train["score"].values.flatten()
X_validation = dev.drop("score", axis=1).values
y_validation = dev["score"].values.flatten()

# local compute
automated_ml_config = AutoMLConfig(
    X=X_train,
    y=y_train,
    X_valid=X_validation,
    y_valid=y_validation,
    verbosity=logging.ERROR,
    **automl_settings  # where the autoML main settings are defined
)

## 3.3 Run Experiment

Run the experiment locally and inspect the results using a widget

In [12]:
experiment = Experiment(ws, "NLP-SS-automl")
local_run = experiment.submit(automated_ml_config, show_output=True)

In [13]:
#local_run.cancel()

The results of the completed run can be visualized in two ways. First, by using a RunDetails widget as shown in the cell below. Second, by accessing the [Azure portal](https://portal.azure.com), selecting your workspace, clicking on _Experiments_ and then selecting the name and run number of the experiment you want to inspect. Both these methods will show the results and duration for each iteration (algorithm tried), a visualization of the results, and information about the run including the compute target, primary metric, etc.

In [14]:
# Inspect the run details using the provided widget
RunDetails(local_run).show()

![](https://nlpbp.blob.core.windows.net/images/autoMLwidget.PNG)

# 4. Deploy Sentence Similarity Model
Deploying an Azure Machine Learning model as a web service creates a REST API. You can send data to this API and receive the prediction returned by the model.
In general, you create a webservice by deploying a model as an image to a Compute Target.

Some of the Compute Targets are: 
1. Azure Container Instance
2. Azure Kubernetes Service
3. Local web service

The general workflow for deploying a model is as follows:
1. Register a model
2. Prepare to deploy
3. Deploy the model to the compute target
4. Test the deployed model (webservice)

In this notebook, we walk you through the process of creating a webservice running on Azure Container Instance by deploying an AutoML model as an image. ACI is typically used for low scale, CPU-based workloads. (You can find more information on deploying and serving models [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where))

## 4.1 Retrieve the Best Model
Now we can identify the model that maximized performance on a given metric (spearman correlation in our case) using the `get_output` method which returns the best_run (AutoMLRun object with information about the experiment) and fitted_model ([Pipeline]((https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-getting-started.ipynb)) object) across all iterations. Overloads on `get_output` allow you to retrieve the best run and fitted model for any logged metric or for a particular iteration. 

The different steps that make up the pipeline can be accessed through `fitted_model.named_steps` and information about data preprocessing is available through `fitted_model.named_steps['datatransformer'].get_featurization_summary()`

In [15]:
best_run, fitted_model = local_run.get_output()

## 4.2 Register the Fitted Model for Deployment

Registering a model means registering one or more files that make up a model. The Machine Learning models are registered in your current Aure Machine Learning Workspace. The model can either come from Azure Machine Learning or another location, such as your local machine. 
Below we show how a model is registered from the results of an experiment run. If neither metric nor iteration are specified in the register_model call, the iteration with the best primary metric is registered.

See other ways to register a model [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where).

In [16]:
description = "AutoML Model"
tags = {"area": "nlp", "type": "sentence similarity automl"}
name = "automl"
model = local_run.register_model(description=description, tags=tags)

print(local_run.model_id)

Registering model AutoML29e4bb98ebest
AutoML29e4bb98ebest


## 4.3 Create an Entry Script
In this section we show an example of an entry script, which is called from the deployed webservice. `score.py` is our entry script. The script must contain:
1. init() - This function loads the model in a global object.
2. run() - This function is used for model prediction. The inputs and outputs to `run()` typically use JSON for serialization and deserilization. 

In [17]:
%%writefile score.py
import pickle
import json
import numpy
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model


def init():
    global model
    model_path = Model.get_model_path(
        model_name="<<modelid>>"
    )  # this name is model.id of model that we want to deploy
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)


def run(rawdata):
    try:
        data = json.loads(rawdata)["data"]
        data = numpy.array(data)
        result = model.predict(data)
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    return json.dumps({"result": result.tolist()})

Overwriting score.py


In [18]:
# Substitute the actual model id in the script file.
script_file_name = "score.py"

with open(script_file_name, "r") as cefr:
    content = cefr.read()

with open(script_file_name, "w") as cefw:
    cefw.write(content.replace("<<modelid>>", local_run.model_id))

## 4.4 Create a YAML File for the Environment

To ensure the fit results are consistent with the training results, the SDK dependency versions need to be the same as the environment that trains the model. The following cells create a file, autoenv.yml, which specifies the dependencies from the run.

In [19]:
ml_run = AutoMLRun(experiment=experiment, run_id=local_run.id)

In [20]:
best_iteration = int(
    best_run.id.split("_")[-1]
)  # get the appended iteration number for the best model
dependencies = ml_run.get_run_sdk_dependencies(iteration=best_iteration)

In [21]:
dependencies

{'azureml-widgets': '1.0.48',
 'azureml-train': '1.0.48',
 'azureml-train-restclients-hyperdrive': '1.0.48',
 'azureml-train-core': '1.0.48',
 'azureml-train-automl': '1.0.48',
 'azureml-telemetry': '1.0.48',
 'azureml-sdk': '1.0.48',
 'azureml-pipeline': '1.0.48',
 'azureml-pipeline-steps': '1.0.48',
 'azureml-pipeline-core': '1.0.48',
 'azureml-mlflow': '1.0.48',
 'azureml-dataprep': '1.1.8',
 'azureml-dataprep-native': '13.0.0',
 'azureml-core': '1.0.48',
 'azureml-automl-core': '1.0.48'}

Add dependencies in the yaml file from the above cell. You must specify the version of "azureml-sdk[automl]" while creating the yaml file.

In [22]:
myenv = CondaDependencies.create(
    conda_packages=["numpy", "scikit-learn==0.21.2", "py-xgboost<=0.80", "pandas==0.24.2"],
    pip_packages=["azureml-sdk[automl]==1.0.48.*"],
    python_version="3.6.8",
)

conda_env_file_name = "automlenv.yml"
myenv.save_to_file(".", conda_env_file_name)

'automlenv.yml'

## 4.5 Create a Container Image

In this step we create a container image which is wrapper containing the entry script, yaml file with package dependencies and the model. The created image is then deployed as a webservice in the next step. This step can take up to 10 minutes and even longer if the model is large.

In [23]:
image_config = ContainerImage.image_configuration(
    execution_script=script_file_name,
    runtime="python",
    conda_file=conda_env_file_name,
    description="Image with automl model",
    tags={"area": "nlp", "type": "sentencesimilarity automl"},
)

image = ContainerImage.create(
    name="automl-image",
    # this is the model object
    models=[model],
    image_config=image_config,
    workspace=ws,
)

image.wait_for_creation(show_output=True)

Creating image
Running...................................................
Succeeded
Image creation operation finished for image automl-image:20, operation "Succeeded"


If the above step fails, then use the below command to see logs

In [24]:
# print(image.image_build_log_uri)

## 4.6 Deploy the Image as a Web Service to Azure Container Instance

Azure Container Instances are mostly used for deploying your models as a web service if one or more of the following conditions are true:
1. You need to quickly deploy and validate your model.
2. You are testing a model that is under development.

To set them up properly, we need to indicate the number of CPU cores and the amount of memory we want to allocate to our web service.

In [25]:
# Set the web service configuration
aci_config = AciWebservice.deploy_configuration(
    cpu_cores=CPU_CORES, memory_gb=MEMORY_GB
)

The final step to deploying our web service is to call `WebService.deploy_from_image()`. This function uses the Docker image and the deployment configuration we created above to perform the following:
1. Deploy the docker image to an Azure Container Instance
2. Call the init() function in our scoring file
3. Provide an HTTP endpoint for scoring calls

The deploy_from_image method requires the following parameters:

1. workspace: the workspace containing the service
2. name: a unique name used to identify the service in the workspace
3. image: a docker image object that contains the environment needed for scoring/inference
4. deployment_config: a configuration object describing the compute type

**Note:** The web service creation can take a few minutes 

In [26]:
# deploy image as web service
aci_service = Webservice.deploy_from_image(
    workspace=ws, name=webservice_name, image=image, deployment_config=aci_config
)

aci_service.wait_for_deployment(show_output=True)
print(aci_service.state)

Creating service
Running............................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


Fetch logs to debug in case of failures.

In [27]:
# print(aci_service.get_logs())

If you want to reuse an existing service versus creating a new one, call the webservice with the name. You can look up all the deployed webservices under deployment in the Azure Portal. Below is an example:

In [28]:
# aci_service = Webservice(workspace=ws, name='<<serive-name>>')

# to use the webservice
# aci_service.run()

## 4.7 Test Deployed Model

Testing the deployed model means running the created webservice. <br>
The deployed model can be tested by passing a list of sentence pairs. The output will be a score between 0 and 5, with 0 indicating no meaning overlap between the sentences and 5 meaning equivalence.

In [29]:
sentences = [
    ["This is sentence1", "This is sentence1"],
    ["A hungry cat.", "A sleeping cat"],
    ["Its summer time ", "Winter is coming"],
]
data = {"data": sentences}
data = json.dumps(data)

In [30]:
# Set up a Timer to see how long the model takes to predict
t = Timer()

t.start()
score = aci_service.run(input_data=data)
t.stop()

print("Time elapsed: {}".format(t))

result = json.loads(score)
try:
    output = result["result"]
    print("Number of samples predicted: {}".format(len(output)))
    print(output)
except:
    print(result["error"])

Time elapsed: 0.3863
Number of samples predicted: 3
[1.8922695576076576, 2.981616317499634, 1.895254173398724]


Finally, we'll calculate the Pearson Correlation on the test set.

**What is Pearson Correlation?**

Our evaluation metric is Pearson correlation ($\rho$) which is a measure of the linear correlation between two variables. The formula for calculating Pearson correlation is as follows:  

$$\rho_{X,Y} = \frac{E[(X-\mu_X)(Y-\mu_Y)]}{\sigma_X \sigma_Y}$$

This metric takes a value in [-1,1] where -1 represents a perfect negative correlation, 1 represents a perfect positive correlation, and 0 represents no correlation. We utilize the Pearson correlation metric as this is the main metric that [SentEval](http://nlpprogress.com/english/semantic_textual_similarity.html), a widely-used evaluation toolkit for evaluation sentence representations, uses for the STS Benchmark dataset.

In [31]:
test_y = test["score"].values.flatten()
test_x = test.drop("score", axis=1).values.tolist()

data = {"data": test_x}
data = json.dumps(data)

In [32]:
# Set up a Timer to see how long the model takes to predict
t = Timer()

t.start()
score = aci_service.run(input_data=data)
t.stop()

print("Time elapsed: {}".format(t))

result = json.loads(score)
try:
    output = result["result"]
    print("Number of samples predicted: {}".format(len(output)))
except:
    print(result["error"])

Time elapsed: 1.1874
Number of samples predicted: 1379


In [33]:
# get Pearson Correlation
pearson = pearsonr(output, test_y)[0]
print(pearson)

sb.glue("pearson_correlation", pearson)

The goal of this notebook is to demonstrate how to use AutoML locally and then deploy the model to Azure Container Instance quickly. The model utilizes the built-in capabilities of AutoML to embed our sentences. The model performance on its own, without tweaking, is not very strong with this particular dataset. For a more advanced model, see [AutoML with Pipelines Deployment AKS](automl_with_pipelines_deployment_aks.ipynb) for much stronger performance on the same task. This notebook utilizes AzureML Pipelines to explicitly embed our sentences using the Google Universal Sentence Encoder (USE) model. For our dataset, the Google USE embeddings result in superior model performance.

## 5. Clean up
Throughout the notebook, we used a workspace and Azure container instances. To get a sense of the cost we incurred, we can refer to this [calculator](https://azure.microsoft.com/en-us/pricing/calculator/). We can also navigate to the [Cost Management + Billing](https://ms.portal.azure.com/#blade/Microsoft_Azure_Billing/ModernBillingMenuBlade/Overview) pane on the portal, click on our subscription ID, and click on the Cost Analysis tab to check our credit usage.
<br><br>
In order not to incur extra costs, let's delete the resources we no longer need.
<br><br>
Once we have verified that our web service works well on ACI, we can delete it. This helps reduce [costs](https://azure.microsoft.com/en-us/pricing/details/container-instances/), since the container group we were paying for no longer exists, and allows us to keep our workspace clean.

In [34]:
# aci_service.delete()

At this point, the main resource we are paying for is the Standard Azure Container Registry (ACR), which contains our Docker image. Details on pricing are available [here](https://azure.microsoft.com/en-us/pricing/details/container-registry/).

We may decide to use our Docker image in a separate ACI or even in an AKS deployment. In that case, we should keep it available in our workspace. However, if we no longer have a use for it, we can delete it.

In [35]:
# docker_image.delete()

If our goal is to continue using our workspace, we should keep it available. On the contrary, if we plan on no longer using it and its associated resources, we can delete it.
<br><br>
Note: Deleting the workspace will delete all the experiments, outputs, models, Docker images, deployments, etc. that we created in that workspace

In [36]:
# ws.delete(delete_dependent_resources=True)
# This deletes our workspace, the container registry, the account storage, Application Insights and the key vault

As mentioned above, Azure Container Instances tend to be used to develop and test deployments. They are typically configured with CPUs, which usually suffice when the number of requests per second is not too high. When working with several instances, we can configure them further by specifically allocating CPU resources to each of them.

For production requirements, i.e. when > 100 requests per second are expected, we recommend deploying models to Azure Kubernetes Service (AKS). It is a convenient infrastructure as it manages hosted Kubernetes environments, and makes it easy to deploy and manage containerized applications without container orchestration expertise. It also supports deployments with CPU clusters and deployments with GPU clusters.For more examples on deployment follow [MachineLearningNotebooks](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/deployment) github repository.


## Next Steps

Check out [AutoML with Pipelines Deployment AKS](automl_with_pipelines_deployment_aks.ipynb) to see how to construct a AzureML Pipeline with an embedding step (using Google Universal Sentence Encoder model) and an AutoMLStep, increasing our Pearson correlation score. Also, this notebooks demonstrates deployment using AKS versus ACI.